In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda lxml --yes

print('Libraries imported.')
import sys
print('System Version', sys.version)

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



openssl-1.1.1g       | 2.1 MB    | #####

# Arcadia and Monterey Park Venues Analysis

In [2]:
CLIENT_ID = 'UODMAXNCJSZVXRDCDYWAENXZQEP1ZDAWLZJDBCSYVSGM2ZD3' # your Foursquare ID
CLIENT_SECRET = 'NQLHSTWUSW3TMCS1OSV4PIZIMT1U1EFUS52DUUKI115D11G5' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UODMAXNCJSZVXRDCDYWAENXZQEP1ZDAWLZJDBCSYVSGM2ZD3
CLIENT_SECRET:NQLHSTWUSW3TMCS1OSV4PIZIMT1U1EFUS52DUUKI115D11G5


# Arcadia Analysis

In [3]:
# Define the latitude and longitude of Westfield Santa Anita from Google (popular place in Arcadia) 
# since Foursquare not able to get the location correctly 
ar_latitude = 34.1344
ar_longitude = -118.0516
radius = 7500
LIMIT =150
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ar_latitude, ar_longitude, VERSION, radius, LIMIT)
#import requests
#results = requests.get(url).json()
#results
url

'https://api.foursquare.com/v2/venues/explore?client_id=UODMAXNCJSZVXRDCDYWAENXZQEP1ZDAWLZJDBCSYVSGM2ZD3&client_secret=NQLHSTWUSW3TMCS1OSV4PIZIMT1U1EFUS52DUUKI115D11G5&ll=34.1344,-118.0516&v=20180604&radius=7500&limit=150'

In [4]:
results = requests.get(url).json()

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address','venue.location.city','venue.location.distance','venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()
#nearby_venues

# Display 20 venues closest to Westfield Santa Anita
arcadia_data = nearby_venues.sort_values('distance')
arcadia_data = arcadia_data.reset_index(drop=True)
arcadia_data.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,address,city,distance,lat,lng
0,Uncle Tetsu Japanese Cheesecake,Dessert Shop,400 S Baldwin Ave,Arcadia,75,34.134954,-118.052070
1,Lady M Cake Boutique,Bakery,400 S Baldwin Ave,Arcadia,76,34.135081,-118.051465
2,Din Tai Fung,Dumpling Restaurant,400 S Baldwin Ave,Arcadia,77,34.135093,-118.051678
3,Nordstrom,Department Store,400 S Baldwin Ave,Arcadia,129,34.135344,-118.052424
4,Chick-fil-A,Fast Food Restaurant,400 S Baldwin Ave,Arcadia,220,34.135790,-118.049897
5,Wood Ranch BBQ & Grill,BBQ Joint,400 S. Baldwin Avenue,Arcadia,261,34.134795,-118.048798
6,Dumpling House,Dumpling Restaurant,921 S Baldwin Ave #A,Arcadia,628,34.129449,-118.054872
7,Kee Wah Bakery,Bakery,1010 S Baldwin Ave,Arcadia,711,34.128488,-118.054539
8,Phoenix Food Boutique,Chinese Restaurant,1108 S Baldwin Ave,Arcadia,845,34.127173,-118.054439
9,Waterfall,Sculpture Garden,NaN,Arcadia,845,34.140627,-118.056859


In [7]:
#Find unique categories
print('There are {} uniques categories.'.format(len(arcadia_data['categories'].unique())))
ac_count = pd.DataFrame(arcadia_data.categories.value_counts().reset_index().values, columns=["Categories", "AC Counts"])
ac_count

There are 45 uniques categories.


,Categories,AC Counts
0,Grocery Store,8
1,Bakery,7
2,Park,5
3,Chinese Restaurant,4
4,Sandwich Place,4
5,Burger Joint,4
6,Mexican Restaurant,4
7,Fast Food Restaurant,4
8,Furniture / Home Store,3
9,Department Store,3


In [8]:
arcadia_vens_map = folium.Map(location=[ar_latitude, ar_longitude], zoom_start=13) # generate map centred around Arcadia

# add a red circle marker to represent the Westfield Santa Anita
folium.features.CircleMarker(
    [34.1344, -118.0516],
    radius=15,
    color='red',
    popup='Westfield Santa Anita',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(arcadia_vens_map)

for lat, lng, label in zip(arcadia_data.lat, arcadia_data.lng, arcadia_data.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(arcadia_vens_map)

# display map
arcadia_vens_map

In [9]:
from folium import plugins

# let's start again with a clean copy of the map of Westfield Santa Anita
arcadia_vens_map = folium.Map(location = [ar_latitude, ar_longitude], zoom_start = 13)

# add a red circle marker to represent the Westfield Santa Anita
folium.features.CircleMarker(
    [34.1344, -118.0516],
    radius=15,
    color='red',
    popup='Westfield Santa Anita',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(arcadia_vens_map)

# instantiate a mark cluster object for the incidents in the dataframe
cluster = plugins.MarkerCluster().add_to(arcadia_vens_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(arcadia_data.lat, arcadia_data.lng, arcadia_data.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(cluster)

# display map
arcadia_vens_map

# Monterey Park Analysis

In [10]:
# Define the latitude and longitude of Atlantic Times Square from Google (popular place in Monterey Park) 
# since Foursquare not able to get the location correctly 
mp_latitude = 34.0689
mp_longitude = -118.1336
radius = 7500
LIMIT =150
mp_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, mp_latitude, mp_longitude, VERSION, radius, LIMIT)
#import requests
#results = requests.get(url).json()
#results
mp_url

'https://api.foursquare.com/v2/venues/explore?client_id=UODMAXNCJSZVXRDCDYWAENXZQEP1ZDAWLZJDBCSYVSGM2ZD3&client_secret=NQLHSTWUSW3TMCS1OSV4PIZIMT1U1EFUS52DUUKI115D11G5&ll=34.0689,-118.1336&v=20180604&radius=7500&limit=150'

In [11]:
mp_results = requests.get(mp_url).json()

In [12]:
mp_venues = mp_results['response']['groups'][0]['items']
    
mp_nearby_venues = json_normalize(mp_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address','venue.location.city','venue.location.distance','venue.location.lat', 'venue.location.lng']
mp_nearby_venues =mp_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
mp_nearby_venues['venue.categories'] = mp_nearby_venues.apply(get_category_type, axis=1)

# clean columns
mp_nearby_venues.columns = [col.split(".")[-1] for col in mp_nearby_venues.columns]

#nearby_venues.head()
#nearby_venues

# Display 20 venues closest to Atlantic Times Square
mp_data = mp_nearby_venues.sort_values('distance')
mp_data = mp_data.reset_index(drop=True)
mp_data.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,address,city,distance,lat,lng
0,Daiso Japan,Convenience Store,500 N Atlantic Blvd Ste 115,Monterey Park,51,34.068458,-118.133768
1,Phoenix Food Boutique,Chinese Restaurant,500 N Atlantic Blvd # C-120,Monterey Park,124,34.067856,-118.134094
2,Huge Tree Pastry,Taiwanese Restaurant,423 N Atlantic Blvd # 105-106,Monterey Park,184,34.067634,-118.134897
3,Hooshik,Dessert Shop,141 N Atlantic Blvd #103,Monterey Park,517,34.064295,-118.134370
4,Tokyo Fried Chicken Co.,Fried Chicken Joint,122 S Atlantic Blvd,Monterey Park,795,34.061752,-118.133433
5,Chengdu Taste,Szechuan Restaurant,828 W Valley Blvd,Alhambra,1012,34.077943,-118.132461
6,Granada Park,Park,2000 W. Hellman Ave,Alhambra,1083,34.067283,-118.145192
7,Angelo's Italian Restaurant & Pizzeria,Italian Restaurant,1540 W Valley Blvd,Alhambra,1103,34.077341,-118.139865
8,P.P. Pop (一條龍),Chinese Restaurant,127 N Garfield Ave,Monterey Park,1142,34.063145,-118.123339
9,Pepe's Mexican Food,Mexican Restaurant,511 W Valley Blvd,Alhambra,1171,34.078526,-118.128465


In [13]:
#Find unique categories
print('There are {} uniques categories.'.format(len(mp_data['categories'].unique())))
#mp_count=mp_data['categories'].value_counts()
mp_count = pd.DataFrame(mp_data.categories.value_counts().reset_index().values, columns=["Categories", "M.P. Counts"])
mp_count

There are 45 uniques categories.


,Categories,M.P. Counts
0,Chinese Restaurant,14
1,Park,6
2,Pizza Place,5
3,Convenience Store,4
4,Mexican Restaurant,4
5,Vietnamese Restaurant,4
6,Sandwich Place,4
7,Fast Food Restaurant,4
8,Bakery,4
9,Italian Restaurant,3


In [14]:
mp_vens_map = folium.Map(location=[mp_latitude, mp_longitude], zoom_start=13) # generate map centred around Monterey Park

# add a red circle marker to represent Atlantic Times Square
folium.features.CircleMarker(
    [34.0689, -118.1336],
    radius=15,
    color='red',
    popup='Atlantic Times Square',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(mp_vens_map)

for lat, lng, label in zip(mp_data.lat, mp_data.lng, mp_data.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(mp_vens_map)

# display map
mp_vens_map

In [15]:
mp_vens_map = folium.Map(location = [mp_latitude, mp_longitude], zoom_start = 13)

# instantiate a mark cluster object for the incidents in the dataframe
cluster = plugins.MarkerCluster().add_to(mp_vens_map)

# add a red circle marker to represent the Atlantic Times Square
folium.features.CircleMarker(
    [34.0689, -118.1336],
    radius=15,
    color='red',
    popup='Atlantic Times Square',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(mp_vens_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(mp_data.lat, mp_data.lng, mp_data.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(cluster)

# display map
mp_vens_map

# Comparing Arcadia and Monterey Park

In [16]:
combine_count = pd.merge(ac_count, mp_count, how='outer', on='Categories')
combine_count = combine_count.replace(np.nan,0)
combine_count

,Categories,AC Counts,M.P. Counts
0,Grocery Store,8,1
1,Bakery,7,4
2,Park,5,6
3,Chinese Restaurant,4,14
4,Sandwich Place,4,4
5,Burger Joint,4,3
6,Mexican Restaurant,4,4
7,Fast Food Restaurant,4,4
8,Furniture / Home Store,3,0
9,Department Store,3,0


In [17]:
# Create a csv file of the above data
combine_count.to_csv('ac vs mp.csv')